In [4]:
import os
import numpy as np
import pandas as pd
import pandas_ta as ta
from pathlib import Path
from datetime import datetime


candlestick_frame = 60*1
pnl_threshold = 10
prev_sma_pc_count = 29


df = pd.read_csv('../../01-11-2019  15-08-2023.csv')
df['sma'] = ta.sma(df['close'], length=7)

ts = list(df['time'])
open = list(df['open'])
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
volume = list(df['volume'])
sma = list(df['sma'])


tot = len(ts)
sma_pc_lst = []
long_runup_lst = []
long_drawdown_lst = []
short_runup_lst = []
short_drawdown_lst = []

for idx in range(tot):
    sma_pc_calc = sma[idx]*100/sma[idx-1]
    sma_pc_lst.append(round(sma_pc_calc-100, 6)) if sma_pc_calc != None else sma_pc_lst.append(0)
    if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
        max_high = max(high[idx:idx+candlestick_frame])
        min_low = min(low[idx:idx+candlestick_frame])
        entry_price = open[idx]
        long_runup_lst.append(round((max_high*100/entry_price)-100, 6))
        long_drawdown_lst.append(round((min_low*100/entry_price)-100, 6))
        short_runup_lst.append(round((entry_price*100/min_low)-100, 6))
        short_drawdown_lst.append(round((entry_price*100/max_high)-100, 6))
    else:
        long_runup_lst.append(0)
        long_drawdown_lst.append(0)
        short_runup_lst.append(0)
        short_drawdown_lst.append(0)     

sma_pc = sma_pc_lst
long_runup = long_runup_lst
long_drawdown = long_drawdown_lst
short_runup = short_runup_lst
short_drawdown = short_drawdown_lst
processed_df = df


long=[]
short=[]
dont_trade=[]

tot = len(ts)

for idx in range(tot):
    if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
        app = {}
        for i in range(prev_sma_pc_count, -1, -1):
            app[f'sma_pc_{i}'] = sma_pc[idx-i]
        app['sma_pc_sum'] = sum(sma_pc[idx-prev_sma_pc_count: idx])
        if long_runup[idx] >= pnl_threshold:
            app['signal']='long'
            long.append(app)
        elif short_runup[idx] >= pnl_threshold:
            app['signal']='short'
            short.append(app)
        else:
            app['signal']='dont_trade'
            dont_trade.append(app)

dataset = []
for idx in range(len(long)):
    dataset.append(long[idx])
    dataset.append(short[idx])
    dataset.append(dont_trade[idx])

print('saving dataset.csv....')
dataset_fn = f'../../dataset.csv'
pd.DataFrame(dataset).to_csv(dataset_fn, index=False)




/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


saving dataset.csv....
